In [59]:
# The code was removed by Watson Studio for sharing.

In [44]:
import numpy as np
import pandas as pd
import requests
import datetime
from pandas.io.json import json_normalize 

date = datetime.date(2012, 4, 10)
df_result_nyc = pd.DataFrame(columns=['astronomy', 'avgtempC', 'avgtempF', 'date', 'hourly', 'maxtempC', 'maxtempF', 'mintempC', 'mintempF', 'sunHour', 'totalSnow_cm', 'uvIndex'])
df_result_tokyo = pd.DataFrame(columns=['astronomy', 'avgtempC', 'avgtempF', 'date', 'hourly', 'maxtempC', 'maxtempF', 'mintempC', 'mintempF', 'sunHour', 'totalSnow_cm', 'uvIndex'])
    
while date < datetime.date(2013, 2, 16):
    previous = date + datetime.timedelta(days=1)
    date = date + datetime.timedelta(days=31)
    #print(str(previous) + ' - ' + str(date))
    #print('http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=ec64962692ca48399ce223327201804&q=New York&format=json&date={start_date}&enddate={end_date}&tp=1'.format(start_date=str(previous), end_date=str(date)))
    result = requests.get('http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=ec64962692ca48399ce223327201804&q=New York&format=json&date={start_date}&enddate={end_date}&tp=1'.format(start_date=str(previous), end_date=str(date)))
    df_result_nyc = df_result_nyc.append(pd.DataFrame(result.json()['data']['weather']))
    result = requests.get('http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=ec64962692ca48399ce223327201804&q=Tokyo&format=json&date={start_date}&enddate={end_date}&tp=1'.format(start_date=str(previous), end_date=str(date)))
    df_result_tokyo = df_result_tokyo.append(pd.DataFrame(result.json()['data']['weather']))
    
project.save_data("weather_nyc.csv", df_result_nyc.to_csv(index=False), overwrite=True)
project.save_data("weather_tokyo.csv", df_result_tokyo.to_csv(index=False), overwrite=True)

In [ ]:
#project.get_assets()
df_nyc = pd.read_csv(project.get_file('dataset_TSMC2014_NYC.csv'))
df_tokyo = pd.read_csv(project.get_file('dataset_TSMC2014_TKY.csv'))